<a href="https://colab.research.google.com/github/harsh-sengar/Computer-Vision-Projects/blob/main/Object_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar
# !wget http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtest_06-Nov-2007.tar

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

In [ ]:
cd /content/gdrive/MyDrive/Research/Object Detection/

In [ ]:
# !mkdir test
# !mv ./VOCtest_06-Nov-2007.tar ./test/
# !tar -xf ./test/VOCtest_06-Nov-2007.tar
# !mv ./VOCdevkit/ ./test/
# !tar -xf VOCtrainval_06-Nov-2007.tar
# #!rm -r VOCdevkit/

In [ ]:
import xml.etree.ElementTree as et
import os
import cv2
import time
import copy
from google.colab.patches import cv2_imshow
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset,DataLoader
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image
from torch.utils.data.sampler import SubsetRandomSampler

In [ ]:
class object_detection(Dataset):
  def __init__(self,label_dict,train_files,data_transforms=None):
    self.data = [] # Bounding box coordinate
    self.labels = [] # Label of the object in that bounding box
    self.paths = [] # Path of the image
    self.data_bbox = {} # Coordinates of all the bounding box  

    for i in train_files:
      tree = et.parse(i)
      path = './VOCdevkit/VOC2007/JPEGImages/'+i[i.rfind('/')+1:].split('.')[0]+'.jpg'
      img = cv2.imread(path)

      bndbox = []

      root = tree.getroot()
      for child in root:
        if child.tag=='object':
          name = ''
          for child1 in child:
            
            if child1.tag=='name':
              name = child1.text
            elif child1.tag=='bndbox':
              #print(name)
              #print(child1[0].text,child1[1].text,child1[2].text,child1[3].text)
              x_min = int(child1[0].text)
              y_min = int(child1[1].text)
              x_max = int(child1[2].text)
              y_max = int(child1[3].text)
              #cv2.rectangle(img,(x_min,y_min),(x_max,y_max),(255,0,0),2)
              
              bndbox.append([x_min,y_min,x_max,y_max])
              self.data.append([x_min,y_min,x_max,y_max])
              self.labels.append(label_dict[name])
              self.paths.append(path)
              #cv2_imshow(img)

      self.data_bbox[path] = bndbox

    # Generate random patch to create a new background class
    j=0
    count=393
    while j<count:
      images = list(self.data_bbox.keys())
      choice = np.random.randint(0,len(images))
      bndbox = self.data_bbox[images[choice]]
      
      img = cv2.imread(images[choice])
    
      h,w,ch = img.shape
      #print(bndbox)
      #print(h,w,ch)
      #print('\n\n')

      
      #for i in bndbox:
        #cv2.rectangle(img,(i[0],i[1]),(i[2],i[3]),(255,0,0),2)
      
      #cv2_imshow(img)
      
      #check = (False,1.5)
      for times in range(6):
        step_size1 = np.round((0.5)*np.random.random_sample()+0.3,1)
        step_size2 = np.round((0.5)*np.random.random_sample()+0.3,1)
        delta_h = int(h*step_size1)
        delta_w = int(w*step_size2)

        #print(len(images),choice)
        y_min = np.random.randint(0,150)
        if y_min+delta_h>h:
          y_max = h
        else:
          y_max = y_min+delta_h
        
        x_min = np.random.randint(0,150)
        if x_min+delta_w>w:
          x_max = w
        else:
          x_max = x_min+delta_w
        check = self.intersection_union(b=[x_min,y_min,x_max,y_max],bndbox=bndbox)
        if check[0]:
          #print([x_min,y_min,x_max,y_max])
          #print(check[1])
          break
      
      if check[0]:
        #img1 = img[y_min:y_max,x_min:x_max]
        #cv2_imshow(img1)
        self.data.append([x_min,y_min,x_max,y_max])
        self.labels.append(0)
        self.paths.append(images[choice])
        j+=1
    
    self.n_samples = len(self.data)


  def __getitem__(self,index):
    
    img = Image.open(self.paths[index])
    x_min = self.data[index][0]
    y_min = self.data[index][1]
    x_max = self.data[index][2]
    y_max = self.data[index][3]
    img1 =  img.crop((x_min,y_min,x_max,y_max))#img[y_min:y_max,x_min:x_max]

    
    #img1 = Image.open('temp.png')

    if data_transforms:
      img1 = data_transforms(img1)

    return img1,self.labels[index]

  def __len__(self):
    return self.n_samples
  
  def intersection_union(self,b,bndbox):
    max_val=0
    for b1 in bndbox:
      x1_min = b[0]
      y1_min = b[1]
      x1_max = b[2]
      y1_max = b[3]
      
      x2_min = b1[0]
      y2_min = b1[1]
      x2_max = b1[2]
      y2_max = b1[3]

      #print(x1_min,y1_min,x1_max,y1_max,x2_min,y2_min,x2_max,y2_max)

      x_dist = max(0,(min(x1_max,x2_max) - max(x1_min,x2_min)))
      y_dist = max(0,(min(y1_max,y2_max)-max(y1_min,y2_min)))

      intersect = x_dist * y_dist
      
      union = (x1_max-x1_min)*(y1_max-y1_min)+(x2_max-x2_min)*(y2_max-y2_min)-intersect

      max_val = max(max_val,intersect/union)
      
      if (intersect/union)>0.2:
        #print(b1)
        #print(intersect/union)
        return False,1.5
    
    return True,max_val

In [ ]:
class object_detection_test(Dataset):
  def __init__(self,label_dict,data_transforms=None):
    self.data = []
    self.labels = []
    self.paths = []
    self.data1 = {} 

    files = []

    for xyz in os.walk('./test/VOCdevkit/VOC2007/Annotations/'):
      path = xyz[0]
      for val in xyz[2]:
        files.append(path+val)


    for i in files:
      tree = et.parse(i)
      path = './test/VOCdevkit/VOC2007/JPEGImages/'+i[i.rfind('/')+1:].split('.')[0]+'.jpg'
      
      #print(type(img))
      #print(img.shape)
      bndbox = []

      root = tree.getroot()
      for child in root:
        if child.tag=='object':
          name = ''
          for child1 in child:
            
            if child1.tag=='name':
              name = child1.text
            elif child1.tag=='bndbox':
              #print(name)
              #classes.add(name)
              #print(child1[0].text,child1[1].text,child1[2].text,child1[3].text)
              x_min = int(child1[0].text)
              y_min = int(child1[1].text)
              x_max = int(child1[2].text)
              y_max = int(child1[3].text)
              #cv2.rectangle(img,(x_min,y_min),(x_max,y_max),(255,0,0),2)
              
              bndbox.append([x_min,y_min,x_max,y_max])
              self.data.append([x_min,y_min,x_max,y_max])
              self.labels.append(label_dict[name])
              self.paths.append(path)
              #cv2_imshow(img1)

      self.data1[path] = bndbox

    j=0
    count=391
    while j<count:
      images = list(self.data1.keys())
      choice = np.random.randint(0,len(images))
      bndbox = self.data1[images[choice]]
      
      img = cv2.imread(images[choice])
    
      h,w,ch = img.shape
      #print(bndbox)
      #print(h,w,ch)
      #print('\n\n')

      
      #for i in bndbox:
        #cv2.rectangle(img,(i[0],i[1]),(i[2],i[3]),(255,0,0),2)
      
      #cv2_imshow(img)
      
      #check = (False,1.5)
      for times in range(6):
        step_size1 = np.round((0.5)*np.random.random_sample()+0.3,1)
        step_size2 = np.round((0.5)*np.random.random_sample()+0.3,1)
        delta_h = int(h*step_size1)
        delta_w = int(w*step_size2)

        #print(len(images),choice)
        y_min = np.random.randint(0,150)
        if y_min+delta_h>h:
          y_max = h
        else:
          y_max = y_min+delta_h
        
        x_min = np.random.randint(0,150)
        if x_min+delta_w>w:
          x_max = w
        else:
          x_max = x_min+delta_w
        check = self.intersection_union(b=[x_min,y_min,x_max,y_max],bndbox=bndbox)
        if check[0]:
          #print([x_min,y_min,x_max,y_max])
          #print(check[1])
          break
      
      if check[0]:
        #img1 = img[y_min:y_max,x_min:x_max]
        #cv2_imshow(img1)
        self.data.append([x_min,y_min,x_max,y_max])
        self.labels.append(0)
        self.paths.append(images[choice])
        j+=1
    
    self.n_samples = len(self.data)


  def __getitem__(self,index):
    
    img = Image.open(self.paths[index])
    x_min = self.data[index][0]
    y_min = self.data[index][1]
    x_max = self.data[index][2]
    y_max = self.data[index][3]
    img1 =  img.crop((x_min,y_min,x_max,y_max))#img[y_min:y_max,x_min:x_max]

    
    #img1 = Image.open('temp.png')

    if data_transforms:
      img1 = data_transforms(img1)

    return img1,self.labels[index]

  def __len__(self):
    return self.n_samples
  
  def intersection_union(self,b,bndbox):
    max_val=0
    for b1 in bndbox:
      x1_min = b[0]
      y1_min = b[1]
      x1_max = b[2]
      y1_max = b[3]
      
      x2_min = b1[0]
      y2_min = b1[1]
      x2_max = b1[2]
      y2_max = b1[3]

      #print(x1_min,y1_min,x1_max,y1_max,x2_min,y2_min,x2_max,y2_max)

      x_dist = max(0,(min(x1_max,x2_max) - max(x1_min,x2_min)))
      y_dist = max(0,(min(y1_max,y2_max)-max(y1_min,y2_min)))

      intersect = x_dist * y_dist
      
      union = (x1_max-x1_min)*(y1_max-y1_min)+(x2_max-x2_min)*(y2_max-y2_min)-intersect

      max_val = max(max_val,intersect/union)
      
      if (intersect/union)>0.2:
        #print(b1)
        #print(intersect/union)
        return False,1.5
    
    return True,max_val

In [ ]:
f = open('./VOCdevkit/VOC2007/ImageSets/Main/train.txt')
train_files = []
for i in f.readlines():
  i = i.split('\n')[0]
  train_files.append('./VOCdevkit/VOC2007/Annotations/'+i+'.xml')

f1 = open('./VOCdevkit/VOC2007/ImageSets/Main/val.txt')
val_files = []
for i in f1.readlines():
  i = i.split('\n')[0]
  val_files.append('./VOCdevkit/VOC2007/Annotations/'+i+'.xml')


print(train_files)
print(val_files)

In [ ]:
train_files_temp = [train_files[0]]
val_files_temp = [val_files[0]]    

In [ ]:
val_files_temp

In [ ]:
data_transforms = transforms.Compose([transforms.Resize((224,224)),
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                          std = [0.229, 0.224, 0.225])])

In [ ]:
labels = {
 'aeroplane': 18,
 'background':0,
 'bicycle': 1,
 'bird': 15,
 'boat': 6,
 'bottle': 16,
 'bus': 14,
 'car': 12,
 'cat': 11,
 'chair': 13,
 'cow': 19,
 'diningtable': 20,
 'dog': 5,
 'horse': 2,
 'motorbike': 10,
 'person': 4,
 'pottedplant': 7,
 'sheep': 8,
 'sofa': 9,
 'train': 3,
 'tvmonitor': 17}
dataset_train = object_detection(label_dict=labels,train_files = train_files_temp,data_transforms=data_transforms)
dataset_val = object_detection(label_dict=labels,train_files = val_files_temp,data_transforms = data_transforms)


In [ ]:
dataset_test = object_detection_test(label_dict=labels,data_transforms=data_transforms)

In [ ]:
first_data = dataset_train[0]
print(first_data[0].shape)
print(first_data[1])
temp = first_data[0].numpy()
inp = temp.transpose((1, 2, 0))
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])
inp = std * inp + mean
inp = np.clip(inp, 0, 1)
plt.imshow(inp)

In [ ]:
first_data = dataset_val[1]
print(first_data[0].shape)
print(first_data[1])
temp = first_data[0].numpy()
inp = temp.transpose((1, 2, 0))
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])
inp = std * inp + mean
inp = np.clip(inp, 0, 1)
plt.imshow(inp)

In [ ]:
batch_size=32
train_loader = DataLoader(dataset_train, batch_size=batch_size, 
                                           shuffle=True,num_workers=8)
validation_loader = DataLoader(dataset_val, batch_size=batch_size,
                                                shuffle=True,num_workers=8)

test_loader = DataLoader(dataset_test,batch_size=batch_size,num_workers=8)
dataloaders = {}
dataloaders['train'] = train_loader
dataloaders['val'] = validation_loader
dataloaders['test'] = test_loader
dataset_sizes = {'train':len(dataset_train),'val':len(dataset_val),'test':len(dataset_test)}

In [ ]:
print(dataset_sizes)


In [ ]:
#validation_split = 0.1
shuffle_dataset = True
random_seed= 42
batch_size=16

# Creating data indices for training and validation splits:
'''dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)
'''
train_loader = DataLoader(dataset_train, batch_size=batch_size, 
                                           shuffle=True,num_workers=8)
validation_loader = DataLoader(dataset_val, batch_size=batch_size,
                                                shuffle=True,num_workers=8)

test_loader = DataLoader(dataset_test ,batch_size=batch_size,num_workers=8)
dataloaders = {}
dataloaders['train'] = train_loader
dataloaders['val'] = validation_loader
dataloaders['test'] = test_loader
dataset_sizes = {'train':len(dataset),'val':len(dataset1),'test':len(dataset2)}


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(dataset_sizes)


In [ ]:
def train_model(model, criterion, optimizer, scheduler,train_loss,val_loss,
                train_acc,val_acc,num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train','train1', 'val']:
            eval = False
            if phase == 'train':
                model.train()  # Set model to training mode
            elif phase=='train1':
                eval = True
                model.eval()
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            if eval:
              phase = 'train'

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                if eval:
                  phase = 'train1'

                '''print('inputs')
                print(inputs.shape)
                print('labels')
                print(labels)'''

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    prob = nn.functional.softmax(outputs)
                    '''print('outputs')
                    print(outputs)
                    print('prob')
                    print(prob)'''
                    _, preds = torch.max(outputs, 1)
                    #print('_')
                    #print(_)
                    #print('preds')
                    #print(preds)
                    loss = criterion(outputs, labels)
                    #print('labels')
                    #print(labels)
                    #print('loss')
                    #print(loss)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                
                running_corrects += torch.sum(preds == labels.data)
                
            if phase == 'train':
                scheduler.step()

            if eval:
              phase = 'train'
            
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
            if eval:
              phase = 'train1'

            if phase!='train':
              print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            if phase == 'train1':
              train_loss.append(epoch_loss)
              train_acc.append(epoch_acc)
            elif phase=='val':
              val_loss.append(epoch_loss)
              val_acc.append(epoch_acc)
            
            time_elapsed = time.time() - since
            print('Epoch complete in {:.0f}m {:.0f}s'.format(
              time_elapsed // 60, time_elapsed % 60))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.fc = nn.Linear(num_ftrs, 21)


model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=50, gamma=0.1)
train_loss = []
val_loss = []
train_acc = []
val_acc = []

In [ ]:
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.0001, momentum=0.9)
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       train_loss,val_loss,train_acc,val_acc,num_epochs=30)
print(train_loss)
print(val_loss)
print(train_acc)
print(val_acc)

In [ ]:
plt.plot(range(80),train_loss,color='blue',label='training')
plt.plot(range(80),val_loss,color='green',label = 'validation')
plt.ylabel('loss')
plt.xlabel('Epochs')
plt.legend()

plt.title('lr=0.01, momentum=0.9) lr_scheduler.StepLR(optimizer_ft, step_size=50, gamma=0.05)')
plt.savefig('loss_epoch_curve.png')
plt.show()

train_acc1 = [float(i) for i in train_acc]
val_acc1 = [float(i) for i in val_acc]

plt.plot(range(80),train_acc1,color='blue',label='training')
plt.plot(range(80),val_acc1,color='green',label = 'validation')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend()

plt.title('lr=0.01, momentum=0.9) lr_scheduler.StepLR(optimizer_ft, step_size=50, gamma=0.05)')
plt.savefig("accuracy_epoch_curve.png")
plt.show()

In [ ]:
phase = 'test'
running_loss = 0.0
since = time.time()
running_corrects = 0
for inputs, labels in dataloaders['test']:
  inputs = inputs.to(device)
  labels = labels.to(device)


                # forward
                # track history if only in train
  with torch.set_grad_enabled(phase == 'train'):
    outputs = model_ft(inputs)
    prob = nn.functional.softmax(outputs)
                    
    _, preds = torch.max(outputs, 1)
                    
    loss = criterion(outputs, labels)
                    
    if phase == 'train':
      loss.backward()
      optimizer.step()

                # statistics
  running_loss += loss.item() * inputs.size(0)
                
  running_corrects += torch.sum(preds == labels.data)
                

epoch_loss = running_loss / dataset_sizes[phase]
epoch_acc = running_corrects.double() / dataset_sizes['test']

print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            
time_elapsed = time.time() - since
print('Epoch complete in {:.0f}m {:.0f}s'.format(

time_elapsed // 60, time_elapsed % 60))

In [ ]:
torch.save(model_ft,"model")
if torch.cuda.is_available():
  model_ft = torch.load("./drive/MyDrive/object_detection/model")
else:
  model_ft = torch.load("./drive/MyDrive/object_detection/model",map_location='cpu')


In [ ]:
labels_dict = {
 'aeroplane': 18,
 'background':0,
 'bicycle': 1,
 'bird': 15,
 'boat': 6,
 'bottle': 16,
 'bus': 14,
 'car': 12,
 'cat': 11,
 'chair': 13,
 'cow': 19,
 'diningtable': 20,
 'dog': 5,
 'horse': 2,
 'motorbike': 10,
 'person': 4,
 'pottedplant': 7,
 'sheep': 8,
 'sofa': 9,
 'train': 3,
 'tvmonitor': 17}
class_names = {}
for i in labels_dict:
  class_names[labels_dict[i]] = i

class_names

In [ ]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

In [ ]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['test']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            prob = nn.functional.softmax(outputs)

            print(prob.shape)
          
            prob,indices = torch.sort(prob,descending=True)
            prob = prob[:,:3]
            print(prob)
            _, preds = torch.max(outputs, 1)

            print(inputs.size())
            preds = preds.to("cpu").numpy()
            labels = labels.to("cpu").numpy()

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                
                
                ax.set_title('predicted: {} {} {}'.format(class_names[preds[j]],prob[j],
                                                        class_names[labels[j]]))
                
                imshow(inputs[j].to("cpu"))

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [ ]:
visualize_model(model_ft)

In [ ]:
data_transforms = torchvision.transforms.Compose([torchvision.transforms.Resize((224,224))
                    ,transforms.ToTensor()]
)

In [ ]:
list(dataset1.data1.keys())[1]

In [ ]:
dataset1.data1['./VOCdevkit/VOC2007/JPEGImages/000009.jpg']

In [ ]:
from PIL import ImageDraw

img = Image.open('./VOCdevkit/VOC2007/JPEGImages/000005.jpg')

w,h = img.size
print(w,h)
width  = [128,256,512,int(w/2),w]
height = [128,256,512,int(h/2),h]

boxes = []
out = display(img,display_id=True)
for s_w in width:
  for s_h in height:
    for y in range(0,h-s_h+1,16):
      for x in range(0,w-s_w+1,16):
        #img2 = copy.deepcopy(img)
        #img1 = img.crop((x,y,x+s_w,y+s_h))
        #img1 = data_transforms(img1)
        #detections.append(img1)
        boxes.append(torch.tensor([x,y,x+s_w,y+s_h]))
        #im = ImageDraw.Draw(img2)
        #im.rectangle([(x,y),(x+s_w,y+s_h)],outline='red')
        #out.update(img2)
        #time.sleep(0.1)
  



In [ ]:
len(boxes)

In [ ]:
#check_detections = torch.stack(detections)
check_boxes = torch.stack(boxes)

filter_detections = []
filter_boxes = []
filter_labels = []
filter_scores = []

from torch.utils.data import TensorDataset
dataset = TensorDataset(check_boxes)
temp1 = DataLoader(dataset,batch_size=32)


model_ft.eval()

with torch.no_grad():
  
  for boxes1 in temp1:
    detections = []
    for i in boxes1[0]:
      a = i[0].item()
      b = i[1].item()
      c = i[2].item()
      d = i[3].item()

      img1 = img.crop((a,b,c,d))
      img1 = data_transforms(img1)
      detections.append(img1)
    
    inputs = torch.stack(detections)
    inputs = inputs.to(device)
    

    outputs = model_ft(inputs)
    prob = nn.functional.softmax(outputs,dim=1)
    

  
    prob,indices = torch.sort(prob,descending=True)
    prob = prob[:,:1]
    #print('prob',prob)

    _, preds = torch.max(outputs, 1)
    #print('_,preds ',_,preds)
    #print('boxes1',boxes1)
    
    preds1 = preds.to("cpu").numpy()
    prob1 = prob.to("cpu").numpy()

    for i in range(len(preds1)):
      
      val = float(prob1[i])
      #print('val',val)
      if val>0.80:
        
        label = class_names[preds1[i]]
        #print('label',label)
        if label !='background':
          #filter_detections.append(inputs[i])
          
          filter_boxes.append(boxes1[0][i])
          filter_labels.append(label)
          filter_scores.append(val)


In [ ]:
temp = [list(i.numpy()) for i in filter_boxes]
len(temp)


In [ ]:
#run = np.array([[4,3,6,6],[4,1,6,3],[2,1,4,3],[2,3,4,6]])
#print(run)
#run = np.delete(run,[1,2],axis=0)
#print(run)

print(overlap[:10])
print(len(idxs))
#print(len(boxes_list))
abc = [i[2] for i in temp1]


In [ ]:
def intersection_union(b,b1,idxs):
  max_val=0
  x1_min = b[0]
  y1_min = b[1]
  x1_max = b[2]
  y1_max = b[3]
  
  x2_min = b1[idxs[:],0]
  y2_min = b1[idxs[:],1]
  x2_max = b1[idxs[:],2]
  y2_max = b1[idxs[:],3]

  #print(x1_min,y1_min,x1_max,y1_max,x2_min,y2_min,x2_max,y2_max)
  

  x1 = np.minimum(x1_max,x2_max)
  x2 = np.maximum(x1_min,x2_min)
  y1 = np.minimum(y1_max,y2_max)
  y2 = np.maximum(y1_min,y2_min)

  #print(x1,x2,y1,y2)

  x_dist = np.maximum(0,(x1 - x2))
  y_dist = np.maximum(0,(y1-y2))

  intersect = x_dist * y_dist
  
  union = (x1_max-x1_min)*(y1_max-y1_min)+(x2_max-x2_min)*(y2_max-y2_min)-intersect

  return intersect/union

'''def nonmax_suppression(filter_boxes,filter_labels,filter_scores):
  
  zipped_tuple = zip(filter_scores,filter_labels,filter_boxes)
  zipped = [x for _,x in sorted(zipped_tuple)]

  print(zipped)'''

#nonmax_suppression(temp,filter_labels,filter_scores)
zipped_tuple = zip(filter_scores,filter_labels,temp)
temp1 = sorted(zipped_tuple,reverse=True)
boxes_list = np.array([i[2] for i in temp1])
pick = []
idxs = np.arange(len(temp1))

while len(idxs)>0:
  pick_i = idxs[0]
  pick.append(pick_i)
  pick_box = boxes_list[pick_i]
  

  overlap = intersection_union(pick_box,boxes_list,idxs)

  idxs = np.delete(idxs,np.concatenate(([0],np.where(overlap>0.2)[0])))

#print('pick',pick)
img2 = copy.deepcopy(img)
#out = display(img2,display_id=True)
counter=0
for i in pick:
  coords = temp1[i][2]
  x1 = coords[0]
  y1 = coords[1]
  x2 = coords[2]
  y2 = coords[3]
  im = ImageDraw.Draw(img2)
  im.rectangle([(x1,y1),(x2,y2)],outline='blue')
  print([(x1,y1),(x2,y2)])
  print(temp1[i][1])
  print(temp1[i][0])
  
'''for i in temp:
  coords = i
  x1 = coords[0]
  y1 = coords[1]
  x2 = coords[2]
  y2 = coords[3]
  im = ImageDraw.Draw(img2)
  im.rectangle([(x1,y1),(x2,y2)],outline='red')
  print([(x1,y1),(x2,y2)])
  #print(temp1[i][1])
  #print(temp1[i][0])
'''
display(img2)

In [ ]:
bndbox = []
tree = et.parse('./VOCdevkit/VOC2007/Annotations/000005.xml')
img2 = Image.open('./VOCdevkit/VOC2007/JPEGImages/000005.jpg')
root = tree.getroot()
for child in root:
  if child.tag=='object':
    name = ''
    for child1 in child:
      
      if child1.tag=='name':
        name = child1.text
      elif child1.tag=='bndbox':
        print(name)
        #classes.add(name)
        #print(child1[0].text,child1[1].text,child1[2].text,child1[3].text)
        x_min = int(child1[0].text)
        y_min = int(child1[1].text)
        x_max = int(child1[2].text)
        y_max = int(child1[3].text)
        #cv2.rectangle(img,(x_min,y_min),(x_max,y_max),(255,0,0),2)
        im = ImageDraw.Draw(img2)
        im.rectangle([(x_min,y_min),(x_max,y_max)],outline='red')
        print([(x_min,y_min),(x_max,y_max)])
        bndbox.append([x_min,y_min,x_max,y_max])

display(img2)

In [ ]:
from PIL import ImageDraw

img = Image.open('./VOCdevkit/VOC2007/JPEGImages/000009.jpg')

w,h = img.size
print(w,h)
width  = [128,256,512,int(w/2),w]
height = [128,256,512,int(h/2),h]

boxes = []
out = display(img,display_id=True)
for s_w in width:
  for s_h in height:
    for y in range(0,h-s_h+1,16):
      for x in range(0,w-s_w+1,16):
        #img2 = copy.deepcopy(img)
        #img1 = img.crop((x,y,x+s_w,y+s_h))
        #img1 = data_transforms(img1)
        #detections.append(img1)
        boxes.append(torch.tensor([x,y,x+s_w,y+s_h]))
        #im = ImageDraw.Draw(img2)
        #im.rectangle([(x,y),(x+s_w,y+s_h)],outline='red')
        #out.update(img2)
        #time.sleep(0.1)

In [ ]:
boxes = {}
for i in val_files:
  
  path = './VOCdevkit/VOC2007/JPEGImages/'+i[i.rfind('/')+1:].split('.')[0]+'.jpg'

  boxes[path] = []
  img = Image.open(path)

  w,h = img.size
  #print(w,h)
  width  = [128,256,512,int(w/2),w]
  height = [128,256,512,int(h/2),h]

  
  #out = display(img,display_id=True)
  for s_w in width:
    for s_h in height:
      for y in range(0,h-s_h+1,16):
        for x in range(0,w-s_w+1,16):
          
          boxes[path].append(torch.tensor([x,y,x+s_w,y+s_h]))


In [ ]:
detections = []
model_ft.eval()
counter=0

for index in boxes:
  check_boxes = torch.stack(boxes[index])

  img = Image.open(index)

    
  filter_boxes = []
  filter_labels = []
  filter_scores = []

  
  dataset_temp = TensorDataset(check_boxes)
  temp1 = DataLoader(dataset_temp,batch_size=32)



  

  with torch.no_grad():
    
    for boxes1 in temp1:
      inputs = []
      for i in boxes1[0]:
        a = i[0].item()
        b = i[1].item()
        c = i[2].item()
        d = i[3].item()

        img1 = img.crop((a,b,c,d))
        img1 = data_transforms(img1)
        inputs.append(img1)
      
      inputs = torch.stack(inputs)
      inputs = inputs.to(device)
      

      outputs = model_ft(inputs)
      prob = nn.functional.softmax(outputs,dim=1)
      

    
      prob,indices = torch.sort(prob,descending=True)
      prob = prob[:,:1]
      #print('prob',prob)

      _, preds = torch.max(outputs, 1)
      #print('_,preds ',_,preds)
      #print('boxes1',boxes1)
      
      preds1 = preds.to("cpu").numpy()
      prob1 = prob.to("cpu").numpy()

      for i in range(len(preds1)):
        
        val = float(prob1[i])
        #print('val',val)
  
        label = class_names[preds1[i]]
        #print('label',label)
        if label !='background':
          #filter_detections.append(inputs[i])
          
          filter_boxes.append(list(boxes1[0][i].numpy()))
          filter_labels.append(label)
          filter_scores.append(val)

  detections.append([index,filter_labels,filter_scores,filter_boxes])



In [ ]:

'''import pickle
with open('detection_list.pkl','wb') as f:
  pickle.dump(detections,f)

detections[0]'''
x = torch.randn(1,2)
x = x.to(device)
print(x)
val = x[0][0]*x[0][1].item()
x[0][0].item()

In [ ]:
!cp ./drive/MyDrive/object_detection/detection.zip ./
!unzip detection.zip

In [ ]:
with open('detection_list.pkl','rb') as f:
  detections = pickle.load(f)

len(detections)

In [ ]:
def intersection_union(b,b1,idxs):
  max_val=0
  x1_min = b[0]
  y1_min = b[1]
  x1_max = b[2]
  y1_max = b[3]
  
  x2_min = b1[idxs[:],0]
  y2_min = b1[idxs[:],1]
  x2_max = b1[idxs[:],2]
  y2_max = b1[idxs[:],3]

  #print(x1_min,y1_min,x1_max,y1_max,x2_min,y2_min,x2_max,y2_max)
  

  x1 = np.minimum(x1_max,x2_max)
  x2 = np.maximum(x1_min,x2_min)
  y1 = np.minimum(y1_max,y2_max)
  y2 = np.maximum(y1_min,y2_min)

  #print(x1,x2,y1,y2)

  x_dist = np.maximum(0,(x1 - x2))
  y_dist = np.maximum(0,(y1-y2))

  intersect = x_dist * y_dist
  
  union = (x1_max-x1_min)*(y1_max-y1_min)+(x2_max-x2_min)*(y2_max-y2_min)-intersect

  return intersect/union

filter_detections = []
for i in detections:
  
  #nonmax_suppression(temp,filter_labels,filter_scores)
  f = i[0]
  zipped_tuple = zip(i[2],i[1],i[3])
  temp1 = sorted(zipped_tuple,reverse=True)
  boxes_list = np.array([j[2] for j in temp1])
  pick = []
  idxs = np.arange(len(temp1))

  while len(idxs)>0:
    pick_i = idxs[0]
    pick.append(pick_i)
    pick_box = boxes_list[pick_i]
    

    overlap = intersection_union(pick_box,boxes_list,idxs)

    idxs = np.delete(idxs,np.concatenate(([0],np.where(overlap>0.4)[0])))

  
  #img2 = copy.deepcopy(img)
  #out = display(img2,display_id=True)
  #counter=0
  filter_boxes = []
  filter_labels = []
  filter_scores = []
  for k in pick:
    '''coords = temp1[k][2]
    x1 = coords[0]
    y1 = coords[1]
    x2 = coords[2]
    y2 = coords[3]
    im = ImageDraw.Draw(img2)
    im.rectangle([(x1,y1),(x2,y2)],outline='blue')
    print([(x1,y1),(x2,y2)])'''
    filter_boxes.append(temp1[k][2])
    filter_labels.append(temp1[k][1])
    filter_scores.append(temp1[k][0])
  
  filter_detections.append([f,filter_labels,filter_scores,filter_boxes])



In [ ]:
filter_detections[0]

In [ ]:
def intersection_union(b,b1):
  max_val=0
  x1_min = b[0]
  y1_min = b[1]
  x1_max = b[2]
  y1_max = b[3]
  
  x2_min = b1[:,0]
  y2_min = b1[:,1]
  x2_max = b1[:,2]
  y2_max = b1[:,3]

  #print(x1_min,y1_min,x1_max,y1_max,x2_min,y2_min,x2_max,y2_max)
  

  x1 = np.minimum(x1_max,x2_max)
  x2 = np.maximum(x1_min,x2_min)
  y1 = np.minimum(y1_max,y2_max)
  y2 = np.maximum(y1_min,y2_min)

  #print(x1,x2,y1,y2)

  x_dist = np.maximum(0,(x1 - x2))
  y_dist = np.maximum(0,(y1-y2))

  intersect = x_dist * y_dist
  
  union = (x1_max-x1_min)*(y1_max-y1_min)+(x2_max-x2_min)*(y2_max-y2_min)-intersect

  return intersect/union

def calc_mAP(predictions,dataset1,iou_thresh,num_classes):

  average_precisions = []
  for class_c in range(1,num_classes):
    filter_predictions = []
    filter_true_box = {}
    total_ground_truth=0
  
    for i in predictions:
      f = i[0]
      pred_class = i[1]
      #pred_boxes = i[3]
    
      for j,lab in enumerate(pred_class):
        if labels_dict[lab]==class_c:          
          filter_predictions.append([f,class_c,i[2][j],i[3][j]])
        
      true_class = dataset1.data1[f][1]
      true_boxes = dataset1.data1[f][0]
      filter_true_box[f] = []

      for j in range(len(true_class)):
        if true_class[j]==class_c:
          filter_true_box[f].append(true_boxes[j])
          total_ground_truth+=1
      
      
    TP = torch.zeros(len(filter_predictions))
    FP = torch.zeros(len(filter_predictions))
    #taken = torch.zeros(len(filter_true_box))
    taken = {}
    for key in filter_true_box:
      taken[key] = torch.zeros(len(filter_true_box[key]))
    
    

    #print('filter_true_box ',filter_true_box,'\ntotal ground truth ',total_ground_truth)
    filter_predictions.sort(key=lambda x:x[2],reverse=True)
    
    for j,prediction in enumerate(filter_predictions):

      f = prediction[0]
      true_boxes = np.array(filter_true_box[f]) 
      #print('true_boxes ',type(true_boxes),true_boxes)
      
      if true_boxes.shape[0]==0:
        FP[j]=1
        continue

      iou_overlap = intersection_union(prediction[3],true_boxes)

      #print(iou_overlap)
      max_iou = np.max(iou_overlap,0)
      idx = np.argmax(iou_overlap)

      if max_iou >= iou_thresh:
        
        if taken[f][idx]==0:
          TP[j]=1
          taken[f][idx]=1
      
        else:
          FP[j] = 1

      else:
        FP[j] = 1
      
    TP_cumsum = torch.cumsum(TP,dim=0)
    FP_cumsum = torch.cumsum(FP,dim=0)
    recalls = TP_cumsum/total_ground_truth
    precisions = TP_cumsum/(TP_cumsum+FP_cumsum)
    precisions = torch.cat((torch.tensor([1]), precisions))
    recalls = torch.cat((torch.tensor([0]), recalls))
    average_precisions.append(torch.trapz(precisions,recalls))
    #print(recalls,'\n', precisions)
    plt.plot(recalls,precisions)
    #plt.xticks(np.arange(0,1.1,0.1))
    #plt.yticks(np.arange(0,1.1,0.1))
    plt.xlabel('recalls')
    plt.ylabel('precisions')
    plt.title(class_names[class_c])
    plt.savefig('{}.png'.format(class_names[class_c]))
    plt.show()

    

  print(sum(average_precisions)/len(average_precisions))


calc_mAP(filter_detections,dataset1,0.3,21)


In [ ]:
!rm *.png

In [ ]:
Image.open('./VOCdevkit/VOC2007/JPEGImages/000005.jpg')

In [ ]:
img = Image.open('./VOCdevkit/VOC2007/JPEGImages/006524.jpg')
data_transforms = torchvision.transforms.Compose([torchvision.transforms.Resize((256)),
                                                  transforms.RandomVerticalFlip(),
                                                  transforms.RandomHorizontalFlip(),
                                                  transforms.RandomAffine(degrees=0,translate=(0.25,0.25))])

#temp = data_transforms(img)
w,h = img.size
diff=0
diff1=0
w1=0
w2=0
h1=0
h2=0
a=0
b=0
while diff<64:
  a = np.random.random()
  a1 = int(a*w)
  b = np.random.random()
  b1 = int(b*w)
  w1 = np.minimum(a1,b1)
  w2 = np.maximum(a1,b1)
  diff = w2-w1
#print(w1,w2,a,b)
while diff1<64:
  a = np.random.random()
  a1 = int(a*h)
  b = np.random.random()
  b1 = int(b*h)
  h1 = np.minimum(a1,b1)
  h2 = np.maximum(a1,b1)
  diff1 = h2-h1
#print(h1,h2,a,b)
img.crop((w1,h1,w2,h2))
